In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [ ]:
# Parameters definition

x_0 = 0 # Initial population
b = 10 # Birth rate
_b = 1 # Number of births
d = 0.1 # Death rate
_d = 1 # Number of deaths
nu = 2000 # Number of samples minus 1 (i.e., nu + 1 equals the number of samples)
nt = 2 # Number of trajectories plus 1
show = True # Plot stuff?
san = True # A simple check

In [ ]:
# Function definition

def SBDP(x_0, b, _b, d, _d, nu, nt, show, san):
    T = [] # A list which stores the lists with the interevent times
    N = [] # A list which stores the lists with the number of individuals
    # j is the index of the current simulation
    for j in range(1, nt):
        tt = 0 # Initial time
        n = [] # A list storing the number of molecules at each given time for the current simulation
        n.append(x_0) # The initial number of molecules
        t = [] # A list storing the interevent times for the current simulation
        t.append(0) # By definition, the first point is 0
        # Here tt < numb, means that numb is the max number of random samples we want to generate for the current simulation
        while tt < nu:
            y1 = np.random.uniform()
            y2 = np.random.uniform()
            z1 = -np.log(y1)/(b+d*n[tt])+t[tt]
            t.append(z1)
            tt += 1
            if b+d*n[tt-1] == 0: # Only the death rate is non-zero in this case!
                n.append(0)
                break
            z2 = (d*n[tt-1])/(b+d*n[tt-1])
            if y2 < z2:
                n.append(n[tt-1]-_d)
            else:
                n.append(n[tt-1]+_b)
        # san = [len(n), len(t)]
        # print(san)
        if show: plt.plot(t, n) # print('Yes') if False else print('No')
        T.append(t) # Store the stochastic realization!
        N.append(n) # Store the stochastic realization!
    if san:
        _san = [len(N), len(T)]
        print(_san)
        print('Over!')
        print(b/d)
        # Check for inconsistencies!
        print([len(i) for i in T])
        print([len(i) for i in N])
    return (N, T)

In [ ]:
(N, T) = SBDP(x_0, b, _b, d, _d, nu, nt, show, san = True)

In [ ]:
DN = [] # The copy number differences! A list of lists!
for n in N:
    DN.append([n[i]-n[i-1] for i in range(1, len(n))])

DT = [] # The time differences! A list of lists!
for t in T:
    DT.append([t[i]-t[i-1] for i in range(1, len(t))])

In [ ]:
# Now we construct a simple function to 'organize' our interevent times, so that we have a 'matching' grid of equally spaced points!
# Please improve (make faster) the function!

def Equidistant(N, T, nu):
    if nu + 1 != len(T[0]):
        print("Something may be wrong!")
    DT = [] # The time differences! A list of lists!
    for t in T:
        DT.append([t[i]-t[i-1] for i in range(1, len(t))]) # The length of the list must be nu - 1
    # sum(DT[0]) == max(T[0]) # It should be 'True' for any index
    MT = [max(i)/nu for i in T] # Mean time
    # We can also try to take the minimum of the time differences for each sample path/function
    pots = [i/j for i in [max(k) for k in T] for j in MT] # This should be equal (close) to nu
    # In general, we can define our grid with equidistant points as a set of nu epochs with interspacing equal to:
    maxi = math.ceil(max([max(i) for i in T])) # mini = min(MT)
    # Actual implementation [Start]
    mini = math.ceil(min([max(i) for i in T]))
    # We could simply round this number to the nearest integer, but it's not a robust solution!
    step = mini/nu
    grid = [i*step for i in range(0, nu+1)]
    NN = []
    for i in range(0, len(T)):
        nn = []
        t = T[i]
        n = N[i]
        for j in range(0, nu+1):
            if j == 0:
                l = 0
            else:
                l = p
            temp = [(grid[j] - k) for k in t[l:] if (grid[j] - k) >= 0]
            p = temp.index(min(temp))
            nn.append(n[p+l])
        NN.append(nn)
    # Actual implementation [End]
    return (NN, grid)

In [ ]:
# Here we check the change of time grid!
(NN, TT) = Equidistant(N, T, nu)

In [ ]:
# This variable, k, should be the index of the simulation one wishes to see!
k = 0
plt.plot(T[k], N[k]) # Original Blue
plt.plot(TT, NN[k]) # Modified Orange

In [ ]:
# This is to better visualize the time evolution after fixing the grid!
# It should work only after running the plot above!
# One letter := Old
# Two letters := New
df = {'N': N[k], 'NN': NN[k], 'T': T[k], 'TT': TT}
a = pd.DataFrame(df)
# a

In [ ]:
# Let's explore the ensemble averages and also the time averages!
df = pd.DataFrame(np.array(NN).transpose(), index = TT)
epoch = 70
_df = df.loc[df.index >= epoch]
ensemble = {'mean': _df.mean(axis = 1), 'std': _df.std(axis = 1)}
timer = {'mean': _df.mean(axis = 0), 'std': _df.std(axis = 0)}

# plt.plot(ensemble['mean'], linewidth = 0.5, linestyle = ':')

In [ ]:
# Variance Estimation
# This one only considers powers of two for splitting!
# For example, if there are 100 data points, then we first estimate variance from 100 points.
# Aftewards, we split the data points in 2, calculate variance in each subset, and take the mean of those two samples.
# We proceed in a similar fashion, now splitting in powers of two!

In [ ]:
def VARE(x):
    # Some definitions!
    l = x.shape[0]
    e = math.floor(math.log2(l))
    # Actual implementation!
    subsets = []
    varas = []
    for _ in range(e):
        step = math.ceil(l/pow(2, _))
        positions = list(range(0, l, step))
        if l not in positions:
            positions.append(l)
        # print(positions)
        subsets.append(step) # subsets.append(len(positions)-1)
        # print(subsets)
        _varas = []
        for i in range(len(positions)-1):
            from_to = positions[i:i+2]
            # print(from_to)
            _x = x.iloc[from_to[0]:from_to[1]]
            # print(_x.shape[0])
            _varas.append(_x.std(axis = 1).mean())
        # print(_varas)
        temp = pd.Series(_varas).mean()
        varas.append(pow(temp, 1))
        # print(varas)
        # print('\n')
        plt.plot(subsets, varas)
    return (subsets, varas)

In [ ]:
VARE(_df)

In [ ]:
math.sqrt(100)

In [ ]:
plt.plot(df, linewidth = 0.5, linestyle = ':')
plt.plot(ensemble['mean'], linewidth = 3, linestyle = '-')
plt.plot(ensemble['mean'] + ensemble['std'], linewidth = 2, linestyle = '-')
plt.plot(ensemble['mean'] - ensemble['std'], linewidth = 2, linestyle = '-')

In [ ]:
print(timer['mean'].mean())
print(ensemble['mean'].mean())
# timer['std'].mean()
print(ensemble['std'].mean())

In [ ]:
a = ensemble['std'].mean()
(a*a)/ensemble['mean'].mean()

In [ ]:
expectation = [10, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
average = [9.95, 100.37, 199.73, 299.84, 400.02, 500.01, 600.3, 701.02, 799.99, 900.01, 1000.12]

In [ ]:
plt.plot(expectation)
plt.plot(average)

In [ ]:
std = [3.17, 10.01, 14.21, 17.25, 19.85, 22.23, 24.57, 26.55, 28.32, 30.14, 31.74]
var = [i*i for i in std]

In [ ]:
plt.plot(average)
plt.plot(var)

In [ ]:
cv = [std[i]/average[i] for i in range(len(std))]
len(cv)

In [ ]:
plt.plot(cv)

In [ ]:
fact = [var[i]/average[i] for i in range(len(std))]
len(fact)

In [ ]:
plt.plot(fact)

In [ ]:
cv = "std/mean"
fano_factor = "variance/mean"

In [ ]:
# Test for Speed!

In [ ]:
temp = np.full(shape = (10, 5), fill_value = 10)

In [ ]:
temp[0, :] = 0

In [ ]:
it = np.nditer(op = temp, flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = 5)
for e in it:
    if it.iterindex == 0:
        _ = e
    else:
        e[...] = e + _
        _ = e

In [ ]:
temp

In [ ]:
s = 1000
t = 500
temp0 = np.random.uniform(size = (s, t))
temp1 = np.full(shape = (s, t), fill_value = 0)

In [ ]:
it = np.nditer(op = [temp0, temp1], flags = ['external_loop', 'buffered'], op_flags = [['readonly'], ['readwrite']], order = 'C', buffersize = t)
for e0, e1 in it:
    p = e0 > 0.5
    e1[p] = e1[p] + 1

In [ ]:
temp1

In [ ]:
# Test for Reality!

In [ ]:
s = 10
t = 1

In [ ]:
# Just do it!